In [0]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


## Wczytujemy biblioteki

In [0]:
import pandas as pd

#modele (algorytmy)
from sklearn.dummy import DummyClassifier           # <== Najprostszy możliwy model 
from sklearn.linear_model import LogisticRegression # <== Regresja logistyczna (liniowa)

#metryka sukcesu
from sklearn.metrics import accuracy_score

In [0]:
cd /content/drive/My Drive/Colab Notebooks/Predykcja płci po imieniu/gender_prediction_by_name

/content/drive/My Drive/Colab Notebooks/Predykcja płci po imieniu/gender_prediction_by_name


## Wczytujemy dane

Dane są w formacie `.csv`, `pandas` umożliwia w jednym wierszu wczytanie danych w formacie `.csv`: `.read_csv()`. 

Po uruchomieniu tej linii `df` będzie zawierać dane wczytane z pliku w postaci tabelarycznej (czyli wiersze i kolumny).

In [0]:
from google.colab import files
uploaded = files.upload()

Saving polish_names.csv to polish_names.csv


In [0]:
df = pd.read_csv("polish_names.csv")
df.head()

,name,gender
0,Abdon,m
1,Abel,m
2,Abercjusz,m
3,Abraham,m
4,Absalon,m


## Sprawdzamy dane

Na początek chcemy wiedzieć bardzo proste rzeczy:
1. Ile jest wierszy (wszystkich obiektów)?
2. Ile jest kolumn (cech obiektów)?
3. Która zmienna jest zmienną docelową (ang. *target variable*)?
4. Jaki problem jest do rozwiązania (klasyfikacja czy regresja)?
5. W przypadku klasyfikacji, ile (dwie czy więcej) i jakie unikalne wartości ma zmienna docelowa?
6. Jak wygląda rozkład unikalnych wartości zmiennej docelowej (czy jest mniej więcej po równo, czy jednak są bardzo popularne/rzadkie klasy)?
7. Czy są brakujące dane?

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 2 columns):
name      1705 non-null object
gender    1705 non-null object
dtypes: object(2)
memory usage: 26.8+ KB


- Druga linia "mówi": `1705 entries`, to jest ilość wierszy (obiektów).
- Trzecia linia "mówi": `total 2 columns`, co oznacza, że mamy 2 kolumny (cechy).
- Następnie mamy informację o każdej kolumnie i liczbę wartości (`non-null`). 
- Jeśli kolumna X ma mniej `non-null` wierszy niż całość, to oznacza, że dla tej cechy mamy brakujące wartości (ang. *missing data*), z którymi trzeba będzie sobie "jakoś" poradzić.
- W naszym przypadku (na początku) wszystko jest bardzo proste. Mamy wszystkie wartości i tylko jedną cechę - **imię**. A druga kolumna to jest zmienna docelowa (eng. *target variable*), czyli czy imię jest **męskie** czy **żeńskie** (tylko dwie wartości, więc klasyfikacja binarna). 
- Ostatnia linia `memory usage` mówi, ile pamięci RAM zużywa, w tym przypadku bardzo mało (jedynie 26.7 KB).

## Jak wyglądają dane?
Podgląd 10 losowych wierszy.

In [0]:
df.sample(10)

,name,gender
23,Adriana,f
1241,Sewer,m
1612,Herkules,m
1648,Kamilla,f
522,Franciszka,f
1198,Rozyna,f
1190,Romuald,m
635,Izabela,f
557,Gertruda,f
307,Czesława,f


- Kolumna `name` zawiera imię
- Kolumna `gender` zawiera płeć, gdzie **`m`** oznacza imię męskie a **`f`** - imię żeńskie

####Jaki jest rozkład imion **męskich** i **żeńskich**?

In [0]:
df['gender'].value_counts()

m    1033
f     672
Name: gender, dtype: int64

## Transformacja słowa -> liczby
Funkcja tożsamościowa

In [0]:
def transform_string_into_number(string):
    return string
    
df['gender'].head().map( transform_string_into_number )

0    m
1    m
2    m
3    m
4    m
Name: gender, dtype: object

In [0]:
def transform_string_into_number(string):
    return int(string == 'm')
    
df['gender'].head().map( transform_string_into_number )

0    1
1    1
2    1
3    1
4    1
Name: gender, dtype: int64

In [0]:
df['target'] = df['gender'].map( lambda x: int(x == 'm') )
df.head(10)

,name,gender,target
0,Abdon,m,1
1,Abel,m,1
2,Abercjusz,m,1
3,Abraham,m,1
4,Absalon,m,1
5,Achacjusz,m,1
6,Achacy,m,1
7,Achilles,m,1
8,Ada,f,0
9,Adalbert,m,1


## Feature engineering
Długość imienia



In [0]:
df['len_name'] = df['name'].map( lambda x: len(x) )

df

,name,gender,target,len_name
0,Abdon,m,1,5
1,Abel,m,1,4
2,Abercjusz,m,1,9
3,Abraham,m,1,7
4,Absalon,m,1,7
...,...,...,...,...
1700,Zbigniewa,f,0,9
1701,Zygfryda,f,0,8
1702,Hermenia,f,0,8
1703,Hermes,m,1,6


##Basic Model

In [0]:
X = df[ ['len_name'] ].values
y = df['target'].values

model = DummyClassifier()
model.fit(X, y)
y_pred = model.predict(X)

***Sprawdzenie wyniku***



In [0]:
df['gender_pred'] = y_pred
df['gender_pred'].value_counts()

1    1038
0     667
Name: gender_pred, dtype: int64

***Weryfikacja błędu***


In [0]:
df[ df.target != y_pred ].shape # błędna odpowiedź

(811, 5)

**Dokładność modelu**

In [0]:
accuracy_score(y, y_pred)

0.5243401759530791

**Losowość modelu**

In [0]:
model = DummyClassifier(random_state=0)
model.fit(X, y)
y_pred = model.predict(X)
accuracy_score(y, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


0.5237536656891496

## Liniowy model


In [0]:
model = LogisticRegression(solver='lbfgs')
model.fit(X, y)
y_pred = model.predict(X)
accuracy_score(y, y_pred)

0.6058651026392962

In [0]:
df['gender_pred'] = y_pred
df['gender_pred'].value_counts()

1    1705
Name: gender_pred, dtype: int64

In [0]:
df[ df.target != y_pred ].shape

(672, 5)

In [0]:
accuracy_score(y, y_pred)

0.6058651026392962

###Podręczna funkcja

In [0]:
def train_and_predict_model(X, y, model, success_metric=accuracy_score):
    model.fit(X, y)
    y_pred = model.predict(X)
    
    print("Distribution:")
    print( pd.Series(y_pred).value_counts() )
    
    return success_metric(y, y_pred)

## Feature engineering
1. Samogłoski

In [0]:
vowels = ['a', 'ą', 'e', 'ę', 'i', 'o', 'u', 'y']

In [0]:
def how_many_vowels(name):
    count = sum( map(lambda x: int(x in vowels), name.lower()) )
    
    return count

In [0]:
df['count_vowels'] = df['name'].map(how_many_vowels)
train_and_predict_model(df[['len_name', 'count_vowels'] ], y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1082
0     623
dtype: int64


0.7143695014662756

2. Samogłoska jako pierwsza litera

In [0]:
def first_is_vowel(name):
    return name.lower()[0] in vowels

In [0]:
df['first_is_vowel'] = df['name'].map(first_is_vowel)

train_and_predict_model(df[['len_name', 'first_is_vowel'] ], y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1705
dtype: int64


0.6058651026392962

3. Długość imienia, ilość samogłosek oraz czy pierwsza litera to samogłoska.

In [0]:
X = df[['len_name', 'count_vowels', 'first_is_vowel'] ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1106
0     599
dtype: int64


0.7296187683284457

####Factorize

In [0]:
pd.factorize(['blue', 'green', 'yellow', 'blue'])

(array([0, 1, 2, 0]), array(['blue', 'green', 'yellow'], dtype=object))

In [0]:
df['first_letter'] = df['name'].map(lambda x: x.lower()[0])
df['first_letter_cnt'] = df['first_letter'].factorize()[0]

X = df[['len_name', 'count_vowels', 'first_is_vowel', 'first_letter_cnt'] ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1106
0     599
dtype: int64


0.7296187683284457

In [0]:
def last_is_vowel(name):
    return name.lower()[-1] in vowels

In [0]:
df['last_is_vowel'] = df['name'].map(last_is_vowel)

train_and_predict_model(df[['len_name', 'last_is_vowel'] ], y, LogisticRegression(solver='lbfgs'))

Distribution:
1    964
0    741
dtype: int64


0.9524926686217009

In [0]:
df['last_letter'] = df['name'].map(lambda x: x.lower()[-1])
df['last_letter_cnt'] = df['last_letter'].factorize()[0]

X = df[['len_name', 'count_vowels', 'last_is_vowel', 'last_letter_cnt'] ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs'))

Distribution:
1    964
0    741
dtype: int64


0.9524926686217009